In [ ]:
import numpy as np
import xarray as xr

from pyrte_rrtmgp import rrtmgp_cloud_optics, rrtmgp_gas_optics
from pyrte_rrtmgp.data_types import (
    AllSkyExampleFiles,
    CloudOpticsFiles,
    GasOpticsFiles,
)
from pyrte_rrtmgp.rte_solver import rte_solve
from pyrte_rrtmgp.utils import (
    compute_clouds,
    compute_profiles,
    load_rrtmgp_file,
)

ncol = 24
nlay = 72

# Create atmospheric profiles and gas concentrations
atmosphere = compute_profiles(300, ncol, nlay)

# Add other gas values
gas_values = {
    "co2": 348e-6,
    "ch4": 1650e-9,
    "n2o": 306e-9,
    "n2": 0.7808,
    "o2": 0.2095,
    "co": 0.0,
}

for gas_name, value in gas_values.items():
    atmosphere[gas_name] = xr.DataArray(
        np.full((ncol, nlay), value), dims=("site", "layer")
    )

cloud_optics_lw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.LW_BND
)

# Calculate cloud properties and merge into the atmosphere dataset
cloud_properties = compute_clouds(
    cloud_optics_lw, atmosphere["pres_layer"], atmosphere["temp_layer"]
)
atmosphere = atmosphere.merge(cloud_properties)

clouds_optical_props = cloud_optics_lw.compute_cloud_optics(
    atmosphere, problem_type="absorption"
)

gas_optics_lw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.LW_G256
)
optical_props = gas_optics_lw.compute_gas_optics(
    atmosphere, problem_type="absorption", add_to_input=False
)
optical_props["surface_emissivity"] = 0.98

# combine optical properties
clouds_optical_props.add_to(optical_props)

fluxes = rte_solve(optical_props, add_to_input=False)

# Load reference data and verify results
ref_data = load_rrtmgp_file(AllSkyExampleFiles.LW_NO_AEROSOL)
assert np.isclose(
    fluxes["lw_flux_up"],
    ref_data["lw_flux_up"].T,
    atol=1e-7,
).all()
assert np.isclose(
    fluxes["lw_flux_down"],
    ref_data["lw_flux_dn"].T,
    atol=1e-7,
).all()

In [ ]:
import numpy as np
import xarray as xr

from pyrte_rrtmgp import rrtmgp_cloud_optics, rrtmgp_gas_optics
from pyrte_rrtmgp.data_types import (
    AllSkyExampleFiles,
    CloudOpticsFiles,
    GasOpticsFiles,
)
from pyrte_rrtmgp.rte_solver import rte_solve
from pyrte_rrtmgp.utils import (
    compute_clouds,
    compute_profiles,
    load_rrtmgp_file,
)

ncol = 24
nlay = 72

# Create atmospheric profiles and gas concentrations
atmosphere = compute_profiles(300, ncol, nlay)

# Add other gas values
gas_values = {
    "co2": 348e-6,
    "ch4": 1650e-9,
    "n2o": 306e-9,
    "n2": 0.7808,
    "o2": 0.2095,
    "co": 0.0,
}

for gas_name, value in gas_values.items():
    atmosphere[gas_name] = xr.DataArray(
        np.full((ncol, nlay), value), dims=("site", "layer")
    )

cloud_optics_sw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.SW_BND
)

# Calculate cloud properties and merge into the atmosphere dataset
cloud_properties = compute_clouds(
    cloud_optics_sw, atmosphere["pres_layer"], atmosphere["temp_layer"]
)

atmosphere = atmosphere.merge(cloud_properties)

clouds_optical_props = cloud_optics_sw.compute_cloud_optics(atmosphere)

gas_optics_sw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.SW_G224
)

# Calculate gas optical properties
optical_props = gas_optics_sw.compute_gas_optics(
    atmosphere, problem_type="two-stream", add_to_input=False
)

clouds_optical_props.add_to(optical_props, delta_scale=True)

# Add SW-specific surface and angle properties
optical_props["surface_albedo_direct"] = 0.06
optical_props["surface_albedo_diffuse"] = 0.06
optical_props["mu0"] = 0.86

fluxes = rte_solve(optical_props, add_to_input=False)

ref_data = load_rrtmgp_file(AllSkyExampleFiles.SW_NO_AEROSOL)
assert np.isclose(
    fluxes["sw_flux_up"],
    ref_data["sw_flux_up"].T,
    atol=1e-7,
).all()
assert np.isclose(
    fluxes["sw_flux_down"],
    ref_data["sw_flux_dn"].T,
    atol=1e-7,
).all()